# Лабораторная работа №1 

### Задание 1

Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).(Документация датасета находится в корне, в файле datasheet.pdf)

### Задание 2

Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Шаг 1. Подготовка данных

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Загрузка данных
df = pd.read_csv("../Base.csv")

print(df)

        fraud_bool  income  name_email_similarity  prev_address_months_count  \
0                1     0.9               0.166828                         -1   
1                1     0.9               0.296286                         -1   
2                1     0.9               0.044985                         -1   
3                1     0.9               0.159511                         -1   
4                1     0.9               0.596414                         -1   
...            ...     ...                    ...                        ...   
999995           0     0.6               0.192631                         -1   
999996           0     0.8               0.322989                        148   
999997           0     0.8               0.879403                         -1   
999998           0     0.9               0.762112                         -1   
999999           0     0.2               0.697452                         -1   

        current_address_months_count  c

### Шаг 2. Разделение данных и нормализация

In [12]:
# Разделение данных на признаки (X) и целевую переменную (y)
X = df.drop("fraud_bool", axis=1)
y = df["fraud_bool"]

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Размер обучающего набора: {X_train.shape}")
print(f"Размер тестового набора: {X_test.shape}")


Размер обучающего набора: (800000, 52)
Размер тестового набора: (200000, 52)


###  Подбор гиперпараметров и обучение модели

In [13]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(2, 11)), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

grid = GridSearchCV(knn, param_grid, refit=True, verbose=3, cv=5)
# Обучение модели
grid.fit(X_train, y_train)

best_params = grid.best_params_
best_knn = grid.best_estimator_

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END ...metric=euclidean, n_neighbors=2;, score=0.989 total time= 4.4min
[CV 2/5] END ...metric=euclidean, n_neighbors=2;, score=0.989 total time= 4.5min
[CV 3/5] END ...metric=euclidean, n_neighbors=2;, score=0.989 total time= 4.7min
[CV 4/5] END ...metric=euclidean, n_neighbors=2;, score=0.989 total time= 4.5min
[CV 5/5] END ...metric=euclidean, n_neighbors=2;, score=0.989 total time= 4.5min
[CV 1/5] END ...metric=euclidean, n_neighbors=3;, score=0.988 total time= 5.1min
[CV 2/5] END ...metric=euclidean, n_neighbors=3;, score=0.988 total time= 4.7min
[CV 3/5] END ...metric=euclidean, n_neighbors=3;, score=0.988 total time= 4.9min
[CV 4/5] END ...metric=euclidean, n_neighbors=3;, score=0.988 total time= 5.3min
[CV 5/5] END ...metric=euclidean, n_neighbors=3;, score=0.988 total time= 5.1min
[CV 1/5] END ...metric=euclidean, n_neighbors=4;, score=0.989 total time= 5.4min
[CV 2/5] END ...metric=euclidean, n_neighbors=4

### Обоснование выбора гиперпараметров

In [ ]:
# Инициализация и обучение модели методом ближайших соседей
predictions = best_knn.predict(X_test)

# Предсказание на тестовых данных
y_pred = knn.predict(X_test)

# Оценка качества модели
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.988575
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99    197776
           1       0.24      0.01      0.02      2224

    accuracy                           0.99    200000
   macro avg       0.61      0.51      0.51    200000
weighted avg       0.98      0.99      0.98    200000

